In [1]:
import basemath_analysis.basemath as bm
import numpy as np

In [2]:
## Generating input data
## The success metric is continuos in this case
## For each visitors converting into a customer, we sample the particular revenue

avg_visitors = 250_000 # average number of visitors per day and variation
cr_A = 0.025 # conversion rate of control
cr_uplift = 0.01 # relative uplift of treatment
days = 90 # number of simulated days
average_revenue_A = 50 # average revenue of each customer in control
average_revenue_B = 52 # average revenue of each customer in treatment

num_visitors = np.abs(np.random.normal(avg_visitors - 1, avg_visitors * 0.1, days).astype(int)) + 1
customers_A = np.random.binomial(n=num_visitors, p=cr_A)
customers_B = np.random.binomial(n=num_visitors, p=cr_A*(1+cr_uplift))
revenues_A = [np.random.exponential(scale = average_revenue_A, size=customers) for customers in customers_A]
revenues_B = [np.random.exponential(scale = average_revenue_B, size=customers) for customers in customers_B]
revenues_delta = (np.array([np.sum(revenue) for revenue in revenues_B])
                  - np.array([np.sum(revenue) for revenue in revenues_A]))

In [3]:
## Running Basemath

# input parameters
mean_A = np.sum(np.concatenate(revenues_A)) / np.sum(num_visitors)
var_A = (
    (np.sum(np.power(np.concatenate(revenues_A) - mean_A, 2))
     + np.sum(num_visitors) - np.sum(customers_A)) * np.power(mean_A, 2)
    / (np.sum(num_visitors) - 1)
)
mde = 0.01
alpha = 0.05
beta = 0.2
exp_name = "test_experiment_ab"

# initialise
bm_test = bm.BaseMathsTest(mean_A, mde, alpha, beta, var_A, seed = exp_name)

# day-by-day evaluation
experiment_state = 0
agg_visitors_seen_before = 0
agg_rev_delta_seen_before = 0
curr_day = 0

for day in range(days):
    experiment_state = bm_test.evaluate_experiment(
        agg_rev_delta_seen_before,
        revenues_delta[day],
        agg_visitors_seen_before,
        num_visitors[day]
    )

    curr_day = day
    agg_visitors_seen_before += num_visitors[day]
    agg_rev_delta_seen_before += revenues_delta[day]
    
    print("Visitors by day {}:\n {} out of {} ({}%)\n\n"
          .format(
              day + 1,
              agg_visitors_seen_before,
              bm_test.required_samples,
              np.round(agg_visitors_seen_before/bm_test.required_samples*100, 2)
          )
         )

    if experiment_state != 0:
        break

# summary
print(f"The experiment evaluation was stopped after {curr_day + 1} days.")
if experiment_state == 1:
    print("Treatment is significantly better than control.")
elif experiment_state == -1:
    print("Treatment is NOT significantly better than control.")
elif experiment_state == 0:
    print("Experiment stopped without conclusion. Not enough data.")
    print(f"{np.sum(num_visitors)} data points per day provided, {bm_test.required_samples} required")

Visitors by day 1:
 251148 out of 17058445 (1.47%)


Visitors by day 2:
 518076 out of 17058445 (3.04%)


Visitors by day 3:
 749367 out of 17058445 (4.39%)


Visitors by day 4:
 969072 out of 17058445 (5.68%)


Visitors by day 5:
 1204580 out of 17058445 (7.06%)


Visitors by day 6:
 1411844 out of 17058445 (8.28%)


Visitors by day 7:
 1660690 out of 17058445 (9.74%)


Visitors by day 8:
 1921039 out of 17058445 (11.26%)


Visitors by day 9:
 2163061 out of 17058445 (12.68%)


Visitors by day 10:
 2415500 out of 17058445 (14.16%)


Visitors by day 11:
 2656379 out of 17058445 (15.57%)


Visitors by day 12:
 2883462 out of 17058445 (16.9%)


Visitors by day 13:
 3169870 out of 17058445 (18.58%)


Visitors by day 14:
 3425615 out of 17058445 (20.08%)


Visitors by day 15:
 3684931 out of 17058445 (21.6%)


Visitors by day 16:
 3947544 out of 17058445 (23.14%)


Visitors by day 17:
 4207966 out of 17058445 (24.67%)


Visitors by day 18:
 4475052 out of 17058445 (26.23%)


Visitors by da